#### API - Flights 

In [1]:
import requests
import pandas as pd

In [2]:
HOST = 'aerodatabox.p.rapidapi.com'
BASE_URL = 'https://aerodatabox.p.rapidapi.com/'
API_TOKEN = '<YOUR TOKEN HERE>'

def get_header():
    return {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": HOST
    }

In [3]:
def get_airport_by_searchterm(searchterm):
    if not searchterm or len(searchterm) < 3: 
        raise Exception('Invalid search term - provide at least 3 characters') 
    
    path = 'airports/search/term'
    querystring = {"q": searchterm.lower(), "limit": "1"}
    res_json = requests.request("GET", BASE_URL+path, headers=get_header(), params=querystring)
    res = res_json.json()
    
    return res['items']

In [4]:
def get_daily_routes_for_airport(icao_code):
    path = f'airports/icao/{icao_code}/stats/routes/daily'
    res_json = requests.request("GET", BASE_URL+path, headers=get_header())
    res = res_json.json()
    
    return res['routes']

In [17]:
def create_flight_overview_for_airport(origin, routes):
    df = pd.json_normalize(routes, record_path=['operators'], 
                           meta=['averageDailyFlights', ['destination', 'icao'], ['destination', 'name'], 
                                 ['destination', 'shortName'], ['destination', 'municipalityName'], 
                                 ['destination','countryCode']], errors='ignore')
    df.rename(columns={'name':'operators.name'}, inplace=True)
    
    df['origin.icao'] = origin['icao']
    df['origin.name'] = origin['name']
    
    first_column = df.pop('origin.icao')
    second_column = df.pop('origin.name')
    df.insert(0, 'origin.icao', first_column)
    df.insert(1, 'origin.name', second_column)
    
    return df

In [18]:
airport = get_airport_by_searchterm(input('Search for Airport: '))
if airport:
    routes = get_daily_routes_for_airport(airport[0]['icao'])
    flight_overview = create_flight_overview_for_airport(airport[0], routes)

Search for Airport: shan


In [20]:
flight_overview

,origin.icao,origin.name,operators.name,averageDailyFlights,destination.icao,destination.name,destination.shortName,destination.municipalityName,destination.countryCode
0,EINN,"Limerick city, Shannon",Aer Lingus,2.43,EGLL,"London Heathrow, London",Heathrow,London,GB
1,EINN,"Limerick city, Shannon",Ryanair,2.02,EGSS,"London Stansted, London",Stansted,London,GB
2,EINN,"Limerick city, Shannon",Aer Lingus,2.02,EGSS,"London Stansted, London",Stansted,London,GB
3,EINN,"Limerick city, Shannon",Ryanair,1.0,EGKK,"London Gatwick, London",Gatwick,London,GB
4,EINN,"Limerick city, Shannon",Aer Lingus,0.73,KBOS,"General Edward Lawrence Logan, Boston",General Edward Lawrence Logan,Boston,US
5,EINN,"Limerick city, Shannon",Ryanair,0.43,EGBB,"Birmingham, Birmingham",NaN,Birmingham,GB
6,EINN,"Limerick city, Shannon",Ryanair,0.43,EGPH,"Edinburgh, Edinburgh",NaN,Edinburgh,GB
7,EINN,"Limerick city, Shannon",Ryanair,0.43,EPKK,"John Paul II Kraków-Balice, Kraków",John Paul II -Balice,Kraków,PL
8,EINN,"Limerick city, Shannon",Ryanair,0.43,EGCC,"Manchester, Manchester",NaN,Manchester,GB
9,EINN,"Limerick city, Shannon",Ryanair,0.43,EPWR,"Copernicus Wrocław, Wrocław",Copernicus,Wrocław,PL
